In [24]:
import os
import openai
openai.api_key = os.environ['OPENAI_API_KEY_']

In [25]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/Mind2Web.pdf']
).load_data()

In [26]:
# Merge into a single document for better retrieval
from llama_index.core import Document

document = Document(
    text='\n\n'.join([doc.text for doc in documents])
)

### Auto-Merge Retriever needs Hierarchical Parser

In [36]:
from llama_index.core.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [28]:
#contains all nodes parent intermediate and leaf nodes
nodes = node_parser.get_nodes_from_documents([document])

### Need only leaf nodes for making index

In [ ]:
from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

### Printing Parent Node

In [ ]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

### Building an Index

In [39]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import ServiceContext
from llama_index.core.settings import Settings


In [40]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002')

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")


/var/folders/4h/c8xgg6gj0_jbc09wh4qlg7sw0000gn/T/ipykernel_16400/4275085086.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  auto_merging_context = ServiceContext.from_defaults(


In [ ]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )

In [41]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="cross-encoder/ms-marco-MiniLM-L-2-v2")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [46]:
auto_merging_response = auto_merging_engine.query(
    'What is the accuracy of the mini2web methodology on unseen websites? And if 38.9% / 39.6% then why is it out of 39.6 and not out of 100'
)

In [44]:
from llama_index.core.response.notebook_utils import display_response

In [47]:
display_response(auto_merging_response)

**`Final Response:`** The accuracy of the mini2web methodology on unseen websites is 38.9% / 39.6%. It is expressed as a percentage out of 39.6 because the evaluation metric used in this context is the step success rate, which is calculated based on the number of successful steps taken by the model during the task.